In [14]:
import cv2
from ultralytics import YOLO
import numpy as np
import joblib
from datetime import datetime, timedelta
import os
import requests
import json
model = YOLO("yolov8n-pose.pt")
knn_model = joblib.load('rf_model.pkl')
knn_scaler = joblib.load('rf_scaler.pkl')
path = "./fall2.mp4"
cap = cv2.VideoCapture(path)
# LINE Notify token
token = 'UU7Ig7LUNtddRq0q3GROs2X0X72957PEVVw1qacyfEd'
notice_time = datetime(1970, 1, 1) 
# URL of the server endpoint
url = "https://196.168.24.94:8081/api/pictures"
# 傳送資料至伺服器
def send_image_with_data(url, image_path, current_time, data):
    keypoints = data.flatten()
    new_list = []
    current_date = current_time.isoformat()
    for i in range(0, len(keypoints) - 1, 2):
        new_list.append(keypoints[i:i + 2].tolist())
    data = {
            "current_date": current_date , # Add the current date to the data
            "keypoints": new_list
        }
    json_data = json.dumps(data)
    headers = {
            "Content-Type": "multipart/form-data"
        }
    with open(image_path, 'rb') as image_file:
            # Create the payload with the image and JSON data
            files = {
                'image': image_file,
                'data': ('data', json_data, 'application/json'),
            }
            response = requests.post(url, files=files, headers=headers)
# 加工資料取得關鍵點座標
def process_image(results):
    r = results[0]
    combined_results = []
    for i in range(len(r.boxes)):
        box_results = []
        for  j  in range(17):
            x, y = r.keypoints.xyn[i][j].numpy()
            box_results.extend([x, y])
            # confidence = r.keypoints.conf[i][j].item()  # Convert tensor to float
        combined_results.append(box_results)
    return combined_results
# line傳送訊息
def send_line_notify(token, image_path=None):
    # Message to send
    message = '有人跌倒了'

    # HTTP headers and data
    headers = { "Authorization": "Bearer " + token }
    data = { 'message': message }
    
    # Send the POST request
    if image_path:
        # Open the image file in binary mode
        with open(image_path, 'rb') as image_file:
            image_data = {'imageFile': image_file}
            response = requests.post("https://notify-api.line.me/api/notify", headers=headers, data=data, files=image_data)
    else:
        response = requests.post("https://notify-api.line.me/api/notify", headers=headers, data=data)

    # Print the response
    print(response.status_code)
    print(response.text)
# 開始影片偵測
while True:
    ret, frame = cap.read()
    if not ret:
        break
    current_time = datetime.now()
    # Perform pose detection
    results = model.track(frame, conf = 0.7) #偵測影片中人體關節點.track 可以將偵測到的人加上id
    annotated_frame = results[0].plot() #.plot 將關鍵點畫到影片上
    r = results[0] # r = 輸出結果 
    # #印出 所有關鍵點的座標位置
    # print(r.keypoints.xyn.numpy().reshape(-1, 2)) 
    # #印出 所有關鍵點的信心值
    # print(r.keypoints.conf.numpy()) 
    #將所有關鍵點的位置與信心值丟入knn/svm/random forest/mlp 模型進行預測(0:沒跌倒/1:失衡/2:跌倒)
    if len(results[0].boxes) >= 1:
        data = np.array(process_image(results)) #  將資料轉成2d numpy array
        data = knn_scaler.transform(data)
        y_pred = knn_model.predict(data)
        print(y_pred)
        if y_pred[0] == 2:
            fall_current_time = current_time
            folder_name = 'Fall_img'
            os.makedirs(folder_name, exist_ok=True)
            time_text = fall_current_time.strftime("%Y-%m-%d %H:%M:%S")
            cv2.imwrite(os.path.join(folder_name, fall_current_time.strftime("%Y-%m-%d_%H%M%S_fall.jpg")), frame)
            print("output save")
            # 將資料傳送至伺服器
            image_path = "./Fall_img/"+fall_current_time.strftime("%Y-%m-%d_%H%M%S_fall.jpg")
            send_image_with_data(url, image_path, current_time, data)
            #通知line，並將圖片送出
            time_difference = notice_time - current_time
            if notice_time == datetime(1970, 1, 1) or time_difference <= timedelta(seconds=1):
                image_path = "./Fall_img/"+fall_current_time.strftime("%Y-%m-%d_%H%M%S_fall.jpg")
                send_line_notify(token, image_path=image_path)
                notice_time = current_time + timedelta(seconds=1)


    # 存進database



    
    cv2.imshow('YOLOv8 Pose Detection', annotated_frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break


cap.release()
cv2.destroyAllWindows()


0: 384x640 (no detections), 201.4ms
Speed: 4.0ms preprocess, 201.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 82.8ms
Speed: 2.0ms preprocess, 82.8ms inference, 0.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 82.6ms
Speed: 2.0ms preprocess, 82.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 80.6ms
Speed: 2.0ms preprocess, 80.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 80.7ms
Speed: 1.0ms preprocess, 80.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 84.5ms
Speed: 1.0ms preprocess, 84.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 81.9ms
Speed: 1.0ms preprocess, 81.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 81.6ms
Speed: 1.0ms preprocess, 81.6ms

ConnectionError: HTTPSConnectionPool(host='196.168.24.94', port=8081): Max retries exceeded with url: /api/pictures (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000020065130B80>: Failed to establish a new connection: [WinError 10060] 連線嘗試失敗，因為連線對象有一段時間並未正確回應，或是連線建立失敗，因為連線的主機無法回應。'))

In [ ]:
import csv
from ultralytics import YOLO
# Load YOLO model
image_path = "C:/Users/User/Desktop/Fall Detection/detect/detected_9281.jpg"
model = YOLO("yolov8n-pose.pt")
results = model(image_path)
r = results[0] 
# by 照片人數去抓每個人的keypoints 
# 照片人數 = len(results[0].boxes)
for i in range(len(results[0].boxes)):
    # print(r.boxes.xywhn[i])
    print(r.keypoints.conf[i]) # keypoint共有17個點,每個點yolo偵測到會給一個信心值
    print(r.keypoints.xyn[i]) # keypoint共有17個點,yolo偵測到的x,y座標並做標準化
    print("-----------------")

# Initialize a list to store the combined (x, y, confidence) tuples
combined_results = []

# Iterate over each keypoint
for i in range(len(r.keypoints.xyn)):
    for  j  in range(17):
        x, y = r.keypoints.xyn[i][j].numpy() # Convert tensor to numpy
        confidence = r.keypoints.conf[i][j].item()  # Convert tensor to float
        combined_results.append((x, y, confidence))
print(combined_results)

In [ ]:
import os
import csv
from ultralytics import YOLO
from PIL import Image
# 圖片偵測存檔範例
save_dir = "C:/Users/leo86/Desktop/fall_dataset/processed_images/"
os.makedirs(save_dir, exist_ok=True)



for i in range(1, 209):
    image_path = f"C:/Users/leo86/Desktop/fall_dataset/images/train/fall{i:03d}.jpg"
    results = model.track(image_path, )
    r = results[0]
    im_array = r.plot()
    im = Image.fromarray(im_array[..., ::-1])
    save_path = os.path.join(save_dir, f"fall{i:03d}.jpg")
    im.save(save_path)

In [ ]:
import os
import csv
from ultralytics import YOLO
from PIL import Image
# 圖片偵測存檔範例
save_dir = "C:/Users/leo86/Desktop/fall_dataset/processed_images/"
os.makedirs(save_dir, exist_ok=True)



for i in range(1, 167):
    image_path = f"C:/Users/leo86/Desktop/fall_dataset/images/train/not fallen{i:03d}.jpg"
    results = model.track(image_path, )
    r = results[0]
    im_array = r.plot()
    im = Image.fromarray(im_array[..., ::-1])
    save_path = os.path.join(save_dir, f"not fallen{i:03d}.jpg")
    im.save(save_path)

In [ ]:
import os

directory = "C:/Users/leo86/Desktop/fall_dataset/processed_images/"

missing_files = []

for i in range(1, 209):
    filename = os.path.join(directory, f"fall{i:03d}.jpg")
    if not os.path.exists(filename):
        missing_files.append(i)

if missing_files:
    print("Missing files:")
    print(missing_files)
else:
    print("All files exist.")


In [4]:
    token = '6quBdcwBnxIbx1XjD8GT1InUIoAn0qmsHKxm8MmWPKC'
    # Message to send
    message = '有人跌倒了'

    # HTTP headers and data
    headers = { "Authorization": "Bearer " + token }
    data = { 'message': message }

    # Send the POST request
    response = requests.post("https://notify-api.line.me/api/notify", headers=headers, data=data)

    # Print the response
    print(response.status_code)
    print(response.text)


200
{"status":200,"message":"ok"}
